# BI & Analytics Integrations

**Cel szkoleniowy:** Integracja Databricks Lakehouse z narzedzniami BI i udostepnianie danych uzytkownikom biznesowym

**Zakres tematyczny:**
- SQL Warehouses: Serverless, Pro, Classic
- Databricks Genie (AI/BI) - natural language queries
- Integracje zewnetrzne: Power BI, Dremio
- Przygotowanie danych dla warstwy BI


## Setup i konfiguracja

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 3 - Integracje i Governance
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
 - Uprawnienia do tworzenia SQL Warehouses (lub dostęp do istniejącego)
 - Unity Catalog

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie jak udostępniać dane z Lakehouse do świata zewnętrznego i użytkowników biznesowych.

**Podstawowe pojęcia:**
- **SQL Warehouse**: Zoptymalizowany silnik obliczeniowy dla zapytań SQL (nie dla kodu Python/Scala).
- **Genie**: Inteligentny asystent danych, który rozumie strukturę tabel i odpowiada na pytania w języku naturalnym.
- **Direct Lake**: Tryb połączenia Power BI, który czyta pliki Parquet bezpośrednio, pomijając warstwę SQL (najszybszy).

## Izolacja per użytkownik

In [ ]:
%run ../00_setup

## Konfiguracja środowiska

In [ ]:
from pyspark.sql import functions as F

# Ustawienie katalogu i schematu
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

display(spark.createDataFrame([
    ("Katalog", CATALOG),
    ("Schemat", GOLD_SCHEMA)
], ["Parametr", "Wartość"]))

## Sekcja 1: Databricks SQL Warehouses

SQL Warehouses to "serce" warstwy BI w Databricks.

### Typy Warehouse'ów

1. **Serverless**: Startuje w sekundy, skaluje się automatycznie. Rekomendowany.
2. **Pro**: Wykorzystuje silnik Photon, ale wymaga dłuższego startu (chyba że jest pula).
3. **Classic**: Starsza architektura (VM-based).

In [ ]:
# Przykład definicji konfiguracji Warehouse (JSON)
# Można to wykorzystać w API Databricks do automatyzacji tworzenia

warehouse_config = {
    "name": "KION_BI_Warehouse_Demo",
    "cluster_size": "2X-Small",
    "min_num_clusters": 1,
    "max_num_clusters": 2,
    "auto_stop_mins": 10,
    "enable_serverless_compute": True,
    "warehouse_type": "PRO",
    "tags": {
        "department": "Sales",
        "cost_center": "1234"
    }
}

# Wyświetlenie konfiguracji jako DataFrame
config_df = spark.createDataFrame([
    ("name", warehouse_config["name"]),
    ("cluster_size", warehouse_config["cluster_size"]),
    ("min_num_clusters", str(warehouse_config["min_num_clusters"])),
    ("max_num_clusters", str(warehouse_config["max_num_clusters"])),
    ("auto_stop_mins", str(warehouse_config["auto_stop_mins"])),
    ("enable_serverless_compute", str(warehouse_config["enable_serverless_compute"])),
    ("warehouse_type", warehouse_config["warehouse_type"]),
    ("tags.department", warehouse_config["tags"]["department"]),
    ("tags.cost_center", warehouse_config["tags"]["cost_center"])
], ["Parametr", "Wartość"])

display(config_df)

## Sekcja 2: Databricks Genie (AI/BI)

Genie pozwala na zadawanie pytań do danych bez znajomości SQL.

### Przygotowanie danych pod Genie

Aby Genie działało dobrze, musimy zadbać o metadane (komentarze do tabel i kolumn).

In [ ]:
# Dodawanie komentarzy do tabeli Gold (kluczowe dla Genie!)
spark.sql(f"""
COMMENT ON TABLE {CATALOG}.{GOLD_SCHEMA}.fact_sales IS 
'Tabela faktów zawierająca transakcje sprzedaży. Zawiera kwoty, daty i klucze obce do wymiarów.'
""")

spark.sql(f"""
COMMENT ON COLUMN {CATALOG}.{GOLD_SCHEMA}.fact_sales.total_amount IS 
'Całkowita wartość zamówienia w PLN (brutto).'
""")

display(spark.createDataFrame([
    ("Metadane", "Zaktualizowane"),
    ("Cel", "Genie będzie teraz lepiej rozumieć te dane")
], ["Status", "Wartość"]))

## Sekcja 3: Integracje Zewnętrzne (Power BI & Dremio)

### Power BI - Direct Lake vs Direct Query

- **Direct Lake**: Power BI Service -> OneLake/Storage (Parquet). Wymaga Fabric lub odpowiedniej konfiguracji.
- **Direct Query**: Power BI -> SQL Warehouse -> Storage.

### Dremio - Unity Catalog Iceberg Endpoint

**Dremio** łączy się z Databricks poprzez **Unity Catalog Iceberg REST Catalog endpoint**. 
Wymaga to włączenia **UniForm (Iceberg reads)** na tabelach Delta.

#### Jak to działa?

```
Dremio → Unity Catalog (Iceberg REST API) → Delta Table z UniForm → Parquet files
```

Delta Lake i Iceberg używają tych samych plików Parquet - UniForm generuje tylko dodatkowe metadane Iceberg, bez kopiowania danych.

#### Wymagania:
1. Tabela zarejestrowana w **Unity Catalog** (managed lub external)
2. **Column mapping** włączony (`delta.columnMapping.mode = 'name'`)
3. **Databricks Runtime 14.3 LTS+** do zapisu
4. Tabela **bez deletion vectors** (lub użyj REORG do ich usunięcia)

In [ ]:
# Krok 1: Włączenie UniForm podczas tworzenia nowej tabeli

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {CATALOG}.{GOLD_SCHEMA}.fact_sales_iceberg
TBLPROPERTIES (
    'delta.columnMapping.mode' = 'name',
    'delta.enableIcebergCompatV2' = 'true',
    'delta.universalFormat.enabledFormats' = 'iceberg'
)
AS SELECT * FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales
""")

In [ ]:
# Krok 2: Włączenie UniForm na istniejącej tabeli (ALTER TABLE)

spark.sql(f"""
ALTER TABLE {CATALOG}.{GOLD_SCHEMA}.dim_customer SET TBLPROPERTIES (
    'delta.columnMapping.mode' = 'name',
    'delta.enableIcebergCompatV2' = 'true',
    'delta.universalFormat.enabledFormats' = 'iceberg'
)
""")

In [ ]:
# Krok 3: Jeśli tabela ma Deletion Vectors - użyj REORG
# REORG usuwa deletion vectors i włącza UniForm w jednym kroku

# spark.sql(f"""
# REORG TABLE {CATALOG}.{GOLD_SCHEMA}.fact_sales 
# APPLY (UPGRADE UNIFORM(ICEBERG_COMPAT_VERSION=2))
# """)

In [ ]:
# Krok 4: Sprawdzenie statusu generowania metadanych Iceberg

result = spark.sql(f"DESCRIBE EXTENDED {CATALOG}.{GOLD_SCHEMA}.dim_customer")
display(result.filter("col_name LIKE '%iceberg%' OR col_name LIKE '%uniform%' OR col_name LIKE '%converted%'"))

#### Konfiguracja Dremio - Unity Catalog Iceberg REST

Dremio łączy się przez **Iceberg REST Catalog API** - **NIE wymaga SQL Warehouse**.

| Połączenie | SQL Warehouse? | Jak działa |
|------------|----------------|------------|
| Power BI / Tableau (JDBC) | TAK | Zapytania przez SQL Warehouse |
| Dremio / Snowflake / Trino (Iceberg REST) | NIE | Bezpośredni dostęp do plików przez API |

**Konfiguracja w Dremio:**

1. **Dodaj źródło** → Iceberg / REST Catalog
2. **Endpoint URI**: 
   ```
   https://<workspace-url>/api/2.1/unity-catalog/iceberg-rest
   ```
3. **Warehouse** (= catalog name): `<uc-catalog-name>` np. `main`
4. **Authentication**: Personal Access Token lub OAuth2 (Service Principal)

Unity Catalog używa **credential vending** - przekazuje tymczasowe credentiale do storage (S3/ADLS), więc Dremio czyta pliki Parquet bezpośrednio.

#### Ręczna synchronizacja metadanych

Jeśli Dremio nie widzi najnowszych danych, wymuś synchronizację:

In [ ]:
# Ręczna synchronizacja metadanych Iceberg (jeśli automatyczna nie zadziałała)
# spark.sql(f"MSCK REPAIR TABLE {CATALOG}.{GOLD_SCHEMA}.dim_customer SYNC METADATA")

#### Ograniczenia UniForm dla Dremio

| Ograniczenie | Opis |
|--------------|------|
| Read-only | Dremio może tylko czytać, nie pisać |
| Deletion Vectors | Muszą być wyłączone (lub użyj REORG) |
| Materialized Views | Nie wspierają UniForm |
| Streaming Tables | Nie wspierają UniForm |
| VOID type | Nie wspierany w Iceberg |

---

### Przygotowanie dedykowanego widoku

Dla narzędzi BI (Dremio, Power BI) dobrą praktyką jest tworzenie widoków, które ukrywają logikę złączeń.

In [ ]:
# Tworzenie widoku raportowego
view_name = "v_sales_summary_bi"

spark.sql(f"""
CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.{view_name} AS
SELECT 
    c.country,
    year(f.order_date) as year,
    month(f.order_date) as month,
    count(distinct f.order_id) as orders_count,
    sum(f.total_amount) as total_revenue
FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales f
JOIN {CATALOG}.{GOLD_SCHEMA}.dim_customer c ON f.customer_id = c.customer_id
GROUP BY 1, 2, 3
""")

display(spark.createDataFrame([
    ("Widok", view_name),
    ("Lokalizacja", f"{CATALOG}.{GOLD_SCHEMA}.{view_name}"),
    ("Status", "Gotowy do podłączenia w BI")
], ["Parametr", "Wartość"]))

In [ ]:
# Weryfikacja widoku
display(spark.table(f"{CATALOG}.{GOLD_SCHEMA}.{view_name}"))

## Best Practices

### Wydajność BI:
- Używaj **Serverless SQL Warehouses** dla najlepszego UX (szybki start).
- Włącz **Photon** (domyślne w Serverless/Pro).
- Stosuj **Materialized Views** dla ciężkich agregacji, jeśli dashboardy działają wolno.

### Governance:
- Nie podłączaj BI bezpośrednio do tabel Silver/Bronze. Używaj tylko **Gold**.
- Używaj dedykowanych **Service Principals** do połączeń z BI, a nie kont osobistych.

## Podsumowanie

1. Skonfigurowaliśmy metadane dla **Genie**.
2. Omówiliśmy typy **SQL Warehouses**.
3. Przygotowaliśmy zoptymalizowany widok dla **Power BI / Dremio**.

## Czyszczenie zasobów

In [ ]:
# spark.sql(f"DROP VIEW IF EXISTS {CATALOG}.{GOLD_SCHEMA}.{view_name}")
display(spark.createDataFrame([("Status", "Zasoby zachowane do dalszych ćwiczeń")], ["Info", "Wartość"]))